This notebook shows how to use thunder's extend submodule to add new operations and custom grad and execution transforms.

In [1]:
from numbers import Number

import thunder
import thunder.torch as ltorch
from thunder.core.devices import DeviceType
from thunder.core.proxies import TensorProxy
from thunder.core.transforms import grad, put_grads, get_grad

import torch
import numpy as np

In [2]:
from thunder.extend import OperatorExecutor, register_executor

In [3]:
# Registers a new operator executor
myex = OperatorExecutor("myex", version="0.1")
register_executor(myex)

myex

In [4]:
# Our operator executor will use the "multimul" function as a new example operator.
#   This function uses NumPy to perform two multiplications of four inputs.
#   This functions very contrived, but will be useful to illustrate the extend submodule's capabilities.
def multimul_impl(
        a: Number | torch.Tensor, 
        b: Number | torch.Tensor,
        c: Number | torch.Tensor,
        d: Number | torch.Tensor,) -> tuple[torch.Tensor, torch.Tensor]:
    return np.multiply(a, b), np.multiply(c, d)

In [5]:
# We can verify that multimul is a valid Python function that operates on PyTorch tensors -- at least PyTorch tensors on the CPU.
a = torch.randn((2, 2))
b = torch.randn((2, 2))
multimul_impl(a, b, a, b)

(tensor([[-0.0687, -0.7520],
         [ 0.1435, -0.1413]]),
 tensor([[-0.0687, -0.7520],
         [ 0.1435, -0.1413]]))

In [6]:
# To let thunder use multimul we need to define how it propagates metadata. This can be done by directly defining a "meta function", 
# of by defining a traceable "like" function that describes what multimul does in terms of existing thunder operations. 
#   The "like" function can be used for metadata propagation AND transforming the new operator, as we'll see below.
#   In this case, the "like" function just describes the two multiplications that multimul performs.
def multimul_like(
        a: Number | TensorProxy, 
        b: Number | TensorProxy,
        c: Number | TensorProxy,
        d: Number | TensorProxy,
):
    return a * b, c * d

In [7]:
# The "register_operator" method of operator executor's returns a "Symbol" object for multimul that can be called directly
#   from compiled thunder code.
multimul = myex.register_operator('multimul', like=multimul_like, fn=multimul_impl)

In [8]:
# Example of calling the new multimul symbol
def foo(a, b, c, d):
    return multimul(a, b, c, d)

cfoo = thunder.compile(foo, executors_list=[myex])
cfoo(a, b, a, b)

(tensor([[-0.0687, -0.7520],
         [ 0.1435, -0.1413]]),
 tensor([[-0.0687, -0.7520],
         [ 0.1435, -0.1413]]))

In [9]:
# The symbol is recorded, like other operations, into thunder's trace
thunder.last_traces(cfoo)[-1]

# Constructed by Delete Last Used (took 0 milliseconds)
import torch

@torch.no_grad()
def foo(a, b, c, d):
  # a: "cpu f32[2, 2]" 
  # b: "cpu f32[2, 2]" 
  # c: "cpu f32[2, 2]" 
  # d: "cpu f32[2, 2]" 
  (t0, t1) = multimul(a, b, c, d)
    # t0 = ltorch.mul(a, b)  # t0: "cpu f32[2, 2]"
      # t0 = prims.mul(a, b)  # t0: "cpu f32[2, 2]"
    # t1 = ltorch.mul(c, d)  # t1: "cpu f32[2, 2]"
      # t1 = prims.mul(c, d)  # t1: "cpu f32[2, 2]"
  del (a, b, c, d)
  return (t0, t1)

In [10]:
# multimul is even differentiable because its "like" function is differentiable
a.requires_grad_(True)
b.requires_grad_(True)

cfoo_grad = grad(cfoo)
cfoo_grad(a, b, a, b)
print(thunder.last_traces(cfoo_grad)[-1])

a.requires_grad_(False)
b.requires_grad_(False)

# Constructed by Delete Last Used (took 0 milliseconds)
import torch

@torch.no_grad()
def foo(a, b, c, d):
  # b: "cpu f32[2, 2]" 
  t10 = torch.full((2, 2), 1, device="cpu", dtype=torch.float32)  # t10: "cpu f32[2, 2]"
    # t10 = ltorch.full((2, 2), 1, device="cpu", dtype=torch.float32)  # t10: "cpu f32[2, 2]"
      # t10 = prims.full((2, 2), 1, device=devices.Device("cpu"), dtype=dtypes.float32)  # t10: "cpu f32[2, 2]"
  t4 = torch.mul(b, t10)  # t4: "cpu f32[2, 2]"
    # t4 = ltorch.mul(b, t10)  # t4: "cpu f32[2, 2]"
      # t4 = prims.mul(b, t10)  # t4: "cpu f32[2, 2]"
  del b
  # a: "cpu f32[2, 2]" 
  t5 = torch.mul(a, t10)  # t5: "cpu f32[2, 2]"
    # t5 = ltorch.mul(a, t10)  # t5: "cpu f32[2, 2]"
      # t5 = prims.mul(a, t10)  # t5: "cpu f32[2, 2]"
  del (a, t10)
  # d: "cpu f32[2, 2]" 
  t11 = torch.full((2, 2), 1, device="cpu", dtype=torch.float32)  # t11: "cpu f32[2, 2]"
    # t11 = ltorch.full((2, 2), 1, device="cpu", dtype=torch.float32)  # t11: "cpu f32[2, 2]"
      # t

tensor([[-0.0768,  0.9247],
        [ 0.3841, -0.4571]])

In [11]:
# We can tell thunder to execute existing operations using multimul by defining a transform
#   from them to multimul, and a "checker" function that returns True when the 
#   transform is valid and False otherwise.

# We can translate mul to multimul by ignoring the second multiplication
def mul_to_multimul(a: Number | TensorProxy, b: Number | TensorProxy) -> TensorProxy:
    result, _ = multimul(a, b, 0, 0)
    return result

# The "checker" function verifies that all inputs are CPU tensors or numbers, because NumPy
#   can't handle other inputs
def mul_to_multimul_checker(a: Number | TensorProxy, b: Number | TensorProxy) -> bool:
    def is_cpu(x: Number | TensorProxy) -> bool:
        if isinstance(a, TensorProxy):
            return a.device.devicetype == DeviceType.CPU
        return True

    return all(is_cpu(x) for x in (a, b))

In [12]:
# The "register_implementation" method describes how to translate mul to multimul
myex.register_implementation(ltorch.mul, checker=mul_to_multimul_checker, execution_transform=mul_to_multimul)

In [13]:
# Verifies the implementation of mul using multimul, and shows the execution transform
def bar(a, b):
    return a * b
cbar = thunder.compile(bar, executors_list=[myex])
cbar(a, b)
thunder.last_traces(cbar)[-1]

# Constructed by Delete Last Used (took 0 milliseconds)
import torch

@torch.no_grad()
def bar(a, b):
  # a: "cpu f32[2, 2]" 
  # b: "cpu f32[2, 2]" 
  (t0, _) = multimul(a, b, 0, 0)
    # t0 = ltorch.mul(a, b)  # t0: "cpu f32[2, 2]"
      # t0 = prims.mul(a, b)  # t0: "cpu f32[2, 2]"
  del (a, b)
  return t0

In [14]:
# Execution transforms happen AFTER semantic transforms like grad, so even when computing the grad
#   of mul (which involves two multiplications to compute the grad) we still see multimul in the
#   execution trace
a.requires_grad_(True)
b.requires_grad_(True)

cbar_grad = grad(cbar)
cbar_grad(a, b)
thunder.last_traces(cbar_grad)[-1]

# Constructed by Delete Last Used (took 0 milliseconds)
import torch

@torch.no_grad()
def bar(a, b):
  # b: "cpu f32[2, 2]" 
  t5 = torch.full((2, 2), 1, device="cpu", dtype=torch.float32)  # t5: "cpu f32[2, 2]"
    # t5 = ltorch.full((2, 2), 1, device="cpu", dtype=torch.float32)  # t5: "cpu f32[2, 2]"
      # t5 = prims.full((2, 2), 1, device=devices.Device("cpu"), dtype=dtypes.float32)  # t5: "cpu f32[2, 2]"
  (t3, _) = multimul(b, t5, 0, 0)
    # t3 = ltorch.mul(b, t5)  # t3: "cpu f32[2, 2]"
      # t3 = prims.mul(b, t5)  # t3: "cpu f32[2, 2]"
  del b
  # a: "cpu f32[2, 2]" 
  (t4, _) = multimul(a, t5, 0, 0)
    # t4 = ltorch.mul(a, t5)  # t4: "cpu f32[2, 2]"
      # t4 = prims.mul(a, t5)  # t4: "cpu f32[2, 2]"
  del (a, t5)
  return [t3, t4]

In [15]:
# In the above grad trace there are two multimuls, and both ignore one of their multiplications.
#   It would be more efficient to perform just one multimul, and we can make this happen
#   by defining a new grad transform for mul that calls multimul once.
#   thunder's grad transforms are defined in a novel way that's not the focus of this notebook,
#   but below we define the grad transform to use multimul.
def mymul_grad(a: TensorProxy, b: TensorProxy) -> TensorProxy:
    fwd = a * b

    g = get_grad(fwd)
    a_grad, b_grad = multimul(b, g, a, g)
    put_grads((a, b), (a_grad, b_grad))

    return fwd

# Re-registers the implementation, including the execution transform and now a grad transform
myex.register_implementation(ltorch.mul, checker=mul_to_multimul_checker, execution_transform=mul_to_multimul, grad_transform=mymul_grad)

In [16]:
# Verifies our new grad transform is used and that a single multimul call is made
cbar_grad = grad(cbar)
cbar_grad(a, b)
thunder.last_traces(cbar_grad)[-1]

# Constructed by Delete Last Used (took 0 milliseconds)
import torch

@torch.no_grad()
def bar(a, b):
  # a: "cpu f32[2, 2]" 
  t5 = torch.full((2, 2), 1, device="cpu", dtype=torch.float32)  # t5: "cpu f32[2, 2]"
    # t5 = ltorch.full((2, 2), 1, device="cpu", dtype=torch.float32)  # t5: "cpu f32[2, 2]"
      # t5 = prims.full((2, 2), 1, device=devices.Device("cpu"), dtype=dtypes.float32)  # t5: "cpu f32[2, 2]"
  # b: "cpu f32[2, 2]" 
  (t3, t4) = multimul(b, t5, a, t5)
    # t3 = ltorch.mul(b, t5)  # t3: "cpu f32[2, 2]"
      # t3 = prims.mul(b, t5)  # t3: "cpu f32[2, 2]"
    # t4 = ltorch.mul(a, t5)  # t4: "cpu f32[2, 2]"
      # t4 = prims.mul(a, t5)  # t4: "cpu f32[2, 2]"
  del (b, t5, a)
  return [t3, t4]

In [17]:
# Some operations may require inputs have particular properties (like be contiguous), or a transform may wish
#   to interleave torch operations with new operations. The transform function supports this. Here
#   we can see an example where the inputs to multimul are made contiguous before it's called
def mul_to_contiguous_multimul(a: Number | TensorProxy, b: Number | TensorProxy) -> TensorProxy:
    a = a.contiguous()
    b = b.contiguous()
    result, _ = multimul(a, b, 0, 0)
    return result

myex.register_implementation(ltorch.mul, checker=mul_to_multimul_checker, execution_transform=mul_to_contiguous_multimul)

In [18]:
# Verifies the new "prologue" for multimul works as expected. Note that the contiguous operations are 
#   executed by PyTorch, and don't have to be executed by your executor
a.requires_grad_(False)
b.requires_grad_(False)

def caz(a, b):
    return a * b
ccaz = thunder.compile(caz, executors_list=[myex])
ccaz(a, b)
thunder.last_traces(ccaz)[-1]

# Constructed by Delete Last Used (took 0 milliseconds)
from torch import Tensor
import torch

@torch.no_grad()
def caz(a, b):
  # a: "cpu f32[2, 2]" 
  # b: "cpu f32[2, 2]" 
  t1 = Tensor.contiguous(a, memory_format=_torch_memory_format_0)  # t1: "cpu f32[2, 2]"
    # t1 = ltorch.contiguous(a, memory_format=_torch_memory_format_0)  # t1: "cpu f32[2, 2]"
      # t1 = prims.stride_order(a, (1, 0))  # t1: "cpu f32[2, 2]"
  del a
  t2 = Tensor.contiguous(b, memory_format=_torch_memory_format_0)  # t2: "cpu f32[2, 2]"
    # t2 = ltorch.contiguous(b, memory_format=_torch_memory_format_0)  # t2: "cpu f32[2, 2]"
      # t2 = prims.stride_order(b, (1, 0))  # t2: "cpu f32[2, 2]"
  del b
  (t0, _) = multimul(t1, t2, 0, 0)
    # t0 = ltorch.mul(t1, t2)  # t0: "cpu f32[2, 2]"
      # t0 = prims.mul(t1, t2)  # t0: "cpu f32[2, 2]"
  del (t1, t2)
  return t0

In [19]:
# NVIDIA's APEX cross-entropy executor is a good example of a real-world operator executor. It defines
#   fast forward and backward functions for torch.nn.functional.cross_entropy. We can see its custom
#   fwd and bwd operations below
# NOTE This cell and the following cells require the apex executor be installed to run properly
dtype = torch.float32
device = 'cuda'
logits = torch.randn([2048, 50257], device=device, dtype=ltorch.to_torch_dtype(dtype), requires_grad=False)
labels = torch.randint(0, 50257, [2048], device=device)

from thunder.executors.apex_entropyex import apex_ex

def foo(logits, labels):
    return torch.nn.functional.cross_entropy(logits, labels, reduction="mean", ignore_index=-1)
cfoo = thunder.compile(foo, executors_list=[apex_ex])

In [20]:
# Shows the forward operation
cfoo(logits, labels)
thunder.last_traces(cfoo)[-1]

# Constructed by Delete Last Used (took 0 milliseconds)
import torch

@torch.no_grad()
def foo(logits, labels):
  # logits: "cuda:0 f32[2048, 50257]" 
  # labels: "cuda:0 i64[2048]" 
  (t18, t20) = apex_cross_entropy(logits, labels, "mean", 0.0)
  del (t20, logits, labels)
  return t18

In [21]:
# Shows APEX's custom forward and backward operations, plus additional PyTorch operations between the two
logits.requires_grad_(True)

cfoo_grad = grad(cfoo)
cfoo_grad(logits, labels)
thunder.last_traces(cfoo_grad)[-1]

# Constructed by Delete Last Used (took 0 milliseconds)
from torch import Tensor
import torch

@torch.no_grad()
def foo(logits, labels):
  # logits: "cuda:0 f32[2048, 50257]" 
  # labels: "cuda:0 i64[2048]" 
  (_, t20) = apex_cross_entropy(logits, labels, "mean", 0.0)
  t25 = Tensor.contiguous(logits, memory_format=_torch_memory_format_0)  # t25: "cuda:0 f32[2048, 50257]"
    # t25 = ltorch.contiguous(logits, memory_format=_torch_memory_format_0)  # t25: "cuda:0 f32[2048, 50257]"
      # t25 = prims.stride_order(logits, (1, 0))  # t25: "cuda:0 f32[2048, 50257]"
  del logits
  t27 = torch.full((), 1, device="cuda:0", dtype=torch.float32)  # t27: "cuda:0 f32[]"
    # t27 = ltorch.full((), 1, device="cuda:0", dtype=torch.float32)  # t27: "cuda:0 f32[]"
      # t27 = prims.full((), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t27: "cuda:0 f32[]"
  t31 = torch.unsqueeze(t27, 0)  # t31: "cuda:0 f32[1]"
    # t31 = ltorch.unsqueeze(t27, 0)  # t31: "cuda:0 f32[1]"
      # t31 =